<a href="https://colab.research.google.com/github/ChaymaeMharzi/NLP-Preprocessing/blob/main/TP3_NLPClassificationDeTexte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d nltkdata/movie-review

  0% 0.00/7.07M [00:00<?, ?B/s]
100% 7.07M/7.07M [00:00<00:00, 92.3MB/s]


In [19]:
!unzip  /content/movie-review

Archive:  /content/movie-review.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [58]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import string

In [59]:
# Chargement du dataset
df = pd.read_csv("/content/movie_review.csv")

In [60]:
# Téléchargement des stop words
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Prétraitement des données textuelles
def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
df["text"] = df["text"].apply(preprocess_text)

In [62]:
# Division des données en ensembles d'entraînement et de test
X = df["text"]
y = df["tag"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [51]:
# Entraînement du modèle Word2Vec
sentences = [text.split() for text in df['text']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

In [63]:
# Vectorisation des reviews de movies
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

In [65]:
def word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index_to_key)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                for tokenized_sentence in corpus]
    return np.array(features)

In [66]:
X = word_vectorizer(sentences, word2vec_model, num_features=100)
# Division des données
X_train, X_test, y_train, y_test = train_test_split(X, df['tag'], test_size=0.2, random_state=42)

In [67]:
# Construction d'un classificateur (exemple avec Logistic Regression)
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [68]:
# Évaluation du modèle
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.5699938195302843
Precision: 0.5717646600898242
Recall: 0.5699938195302843
F1 Score: 0.5644612308830462


In [69]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.5699938195302843
Classification Report:
               precision    recall  f1-score   support

         neg       0.58      0.46      0.51      6371
         pos       0.56      0.68      0.62      6573

    accuracy                           0.57     12944
   macro avg       0.57      0.57      0.56     12944
weighted avg       0.57      0.57      0.56     12944

